In [ ]:
!pip install Flask pyngrok line-bot-sdk requests openai==1.55.3 --quiet

In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
openai_api_key = userdata.get('OPENAI_API_KEY')
port = 5051
print(line_channel_access_token)
print("====")
print(line_channel_secret)

In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(port, name="linebot_tunnel")
print(tunnel.public_url)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)
def get_completion(prompt):
    messages = [{"role": "system", "content": "以下用繁體中文台灣用語回答"},
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=256

    )
    return response.choices[0].message.content

In [ ]:
result = get_completion("簡介東吳大學")
print(result)

In [ ]:
messages = []

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            messages.append(prompt)
            prompts = " ".join(messages)
            reply_text = get_completion(prompts)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )
            messages.append(reply_text)
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )


@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)